<a href="https://colab.research.google.com/github/AshritaVaka/html-portfolio/blob/main/roi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, exposure, morphology, measure
from skimage import io, color, filters, segmentation, morphology
from skimage.segmentation import active_contour
import os
import glob



image_folder = "/gdrive/MyDrive/Colab Notebooks/Data Set for Breast Cancer Patients/extracted images/imgext29_0"
output_folder = "/gdrive/MyDrive/Colab Notebooks/Data Set for Breast Cancer Patients/roi/imgroi29_0"

i = 0

os.makedirs(output_folder, exist_ok=True)

# Iterate over all the images in the specified folder and extract the Haralick texture features
for image_path in glob.glob(os.path.join(image_folder, "*.png")):
  # Load histopathology image
  image = io.imread(image_path)

  # Function for preprocessing an image
  def preprocess_image(image, target_size=(512, 512)):

    # Ensure the image is in RGB format
    if image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

    # Resize the image for consistent analysis
    resized_image = cv2.resize(image, target_size)

    # Color normalization (Reinhard)
    img_lab = cv2.cvtColor(resized_image, cv2.COLOR_RGB2LAB)
    img_lab_planes = cv2.split(img_lab)

    # Convert the tuple to a list so that we can modify it
    img_lab_planes = list(img_lab_planes)

    # Apply histogram equalization to the L channel
    img_lab_planes[0] = cv2.equalizeHist(img_lab_planes[0])

    # Merge the modified channels back to LAB format
    img_lab_normalized = cv2.merge(img_lab_planes)
    normalized_image = cv2.cvtColor(img_lab_normalized, cv2.COLOR_LAB2RGB)

    # Convert to grayscale
    grayscale_image = color.rgb2gray(normalized_image)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = exposure.equalize_adapthist(grayscale_image, clip_limit=0.03)

    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(clahe, (5, 5), 0)

    return blurred_image

  # Preprocess the image
  preprocessed_image = preprocess_image(image)

  # Display the preprocessed image
  # plt.figure(figsize=(8, 8))
  # plt.title('Preprocessed Image')
  # plt.imshow(preprocessed_image, cmap='gray')
  # plt.axis('off')
  # plt.show()



  # Save the preprocessed image as 'output.png'
  output_path = 'output.png'
  cv2.imwrite(output_path, (preprocessed_image * 255).astype(np.uint8))

  print(f'Preprocessed image saved as {output_path}')

  image_path = 'output.png'
  gray_image = io.imread(image_path)

  # Apply edge detection to find contours
  edges = filters.sobel(gray_image)

  # Perform morphological operations to enhance edges (optional)
  selem = morphology.disk(2)
  edges = morphology.closing(edges, selem)

  # Initialize the contour using a rectangle or ellipse
  # Replace these coordinates with the desired initial contour
  # For example, for a rectangle, use [(x0, y0), (x1, y1), (x2, y2), (x3, y3)]
  initial_contour = np.array([(100, 100), (400, 100), (400, 400), (100, 400)])

  # Create an active contour model using GVF
  snake = active_contour(edges, initial_contour, alpha=0.015, beta=10.0, gamma=0.001)

  # Display the original image with the extracted ROI
  # fig, ax = plt.subplots(figsize=(8, 8))
  # ax.imshow(gray_image, cmap=plt.cm.gray)
  # ax.plot(initial_contour[:, 0], initial_contour[:, 1], '--r', lw=3)
  # ax.plot(snake[:, 0], snake[:, 1], '-b', lw=3)
  # ax.set_xticks([]), ax.set_yticks([])
  # ax.set_title('GVF Active Contour')

  # Save the result as 'output2.png' in the local storage
  output_path = 'output2.png'
  plt.savefig(output_path, bbox_inches='tight', pad_inches=0, dpi=300)

  # Show the result
  #plt.show()

  # Load the preprocessed image
  image_path = 'output.png'
  image = cv2.imread(image_path)

  # Convert the image to LAB color space
  lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

  # Reshape the LAB image to a 2D array of pixels
  pixels = lab_image.reshape((-1, 3))
  pixels = np.float32(pixels)

  # Define the number of clusters (e.g., 2 for ROI and background)
  num_clusters = 2

  # Perform k-means clustering
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
  _, labels, centers = cv2.kmeans(pixels, num_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

  # Reshape the labels to the shape of the original image
  labels = labels.reshape(image.shape[:2])

  # Assign the ROI label (cluster) based on the majority of pixels in the ROI
  roi_label = 1 if np.sum(labels == 1) > np.sum(labels == 0) else 0

  # Create a mask for the ROI
  roi_mask = np.uint8(labels == roi_label)

  # Extract the ROI from the original image using the mask
  roi = cv2.bitwise_and(image, image, mask=roi_mask)

  # Display the original image and the extracted ROI
  # plt.figure(figsize=(12, 6))
  # plt.subplot(121)
  # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  # plt.title('Original Image')
  # plt.axis('off')

  # plt.subplot(122)
  # plt.imshow(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
  # plt.title('Extracted ROI')
  # plt.axis('off')

  plt.tight_layout()

  # Save the extracted ROI as 'output3.png'
  output_path = 'output3.png'
  cv2.imwrite(output_path, roi)

  print(f'Extracted ROI saved as {output_path}')

  # Show the result
  #plt.show()
  output_image_path = os.path.join(output_folder, f"image_{i}.png")
  cv2.imwrite(output_image_path, cv2.cvtColor(roi, cv2.COLOR_RGB2BGR))

  i += 1

print("done")

In [ ]:
#path that contains folder you want to copy
%cd /gdrive/My Drive/Colab Notebooks
%cp -av /gdrive/MyDrive/Colab Notebooks/roi/ imgroi

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-a7c5d7188481>", line 2, in <cell line: 2>
    get_ipython().run_line_magic('cd', '/gdrive/My Drive/Colab Notebooks')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/cor

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-a7c5d7188481>", line 2, in <cell line: 2>
    get_ipython().run_line_magic('cd', '/gdrive/My Drive/Colab Notebooks')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/cor